In [56]:
# Dependencies
import os
import pandas as pd
import requests as req
import time
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [57]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [58]:
# options = Options()
# options.add_argument("start-maximized")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
# driver = webdriver.Chrome(chrome_options=options, executable_path='/usr/local/bin/chromedriver')
# driver.get('https://www.google.com')
# browser = Browser('chrome', **executable_path, headless=False)


In [59]:
options = Options()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', chrome_options=options,**executable_path, headless=False)


In [63]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)
html = browser.html

soup = bs(html, "html.parser")

# NASA Mars News

In [64]:
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print("-----------")
print(news_p)

Media Get a Close-Up of NASA's Mars 2020 Rover
-----------
The clean room at NASA's Jet Propulsion Laboratory was open to the media to see NASA's next Mars explorer before it leaves for Florida in preparation for a summertime launch.


# JPL Mars Space Images - Featured Image

In [65]:
mars_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(mars_image_url)
image_html = browser.html
soup = bs(image_html, 'html.parser')

In [66]:
base_url = 'https://www.jpl.nasa.gov'

image_url = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

In [67]:
full_image_url = base_url + image_url

full_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18886-1920x1200.jpg'

# Mars Weather

In [17]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [18]:
weather_html = browser.html
soup = bs(weather_html,'html.parser')

In [19]:
recent_tweets = soup.find_all('div', class_='js-tweet-text-container')

for tweet in recent_tweets: 
    weather_tweet = tweet.find('p').text
    if 'InSight' and 'sol' in weather_tweet:
        print(weather_tweet)
        break
    else: 
        pass

InSight sol 385 (2019-12-27) low -96.4ºC (-141.4ºF) high -18.6ºC (-1.6ºF)
winds from the SSE at 6.4 m/s (14.3 mph) gusting to 21.2 m/s (47.5 mph)
pressure at 6.50 hPapic.twitter.com/O06hld1B8X


# Mars Facts

In [27]:
facts_url = 'https://space-facts.com/mars/'
browser.visit(facts_url)

table = pd.read_html(facts_url)

In [33]:
facts_table = table[0]

facts_table.columns = ['Fact Description','Value']
facts_table

,Fact Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [34]:
facts_table = facts_table.set_index('Fact Description')
facts_table

,Value
Fact Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [35]:
facts_table.to_html('mars_table.html')

# Mars Hemispheres

In [52]:
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [53]:
hemispheres_html = browser.html
soup = bs(hemispheres_html, 'html.parser')

In [54]:
items_url = soup.find_all('div', class_='item')
hemi_image_urls = []

In [55]:
base_url = 'https://astrogeology.usgs.gov'

for item in items_url: 
    title = item.find('h3').text
    image_url = item.find('a', class_='itemLink product-item')['href'] 
    browser.visit(base_url + image_url)
    image_html = browser.html
    soup = bs(image_html, 'html.parser')
    full_url = base_url + soup.find('img', class_='wide-image')['src'] 
    hemi_image_urls.append({"title" : title, "img_url" : full_url})
    
print(hemi_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
